<a href="https://colab.research.google.com/github/StonecoldLi/Kaggle_Contest_Samli/blob/main/Natural%20Language%20Processing%20with%20Disaster%20Tweets/Bert_finetune_official_version_basedonhuggingface_bertbaseuncased_keywordtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri May 26 12:41:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q kaggle

读取竞赛数据集

In [3]:
#上传凭证
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jfli318","key":"475ac384f8639d8bb024bc7891fec449"}'}

In [4]:
#create a kaggle folder
!mkdir ~/.kaggle

In [5]:
#copy the kaggle.json to folder created
!cp kaggle.json ~/.kaggle/

In [6]:
# permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle config set -n path -v .

- path is now set to: .


In [8]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 139MB/s]


In [9]:
!unzip '/content/competitions/nlp-getting-started/nlp-getting-started.zip' -d '.'

Archive:  /content/competitions/nlp-getting-started/nlp-getting-started.zip
  inflating: ./sample_submission.csv  
  inflating: ./test.csv              
  inflating: ./train.csv             


## Install the basic libraries

In [10]:
!pip install transformers==4.28.0 datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━

In [11]:
from huggingface_hub import notebook_login

notebook_login()

## Set Dataset
  - set the dataset into 2 columns 1 is text, 1 is label
  - we suppose that Location and Keyword make no sense in this task at first

In [12]:
import pandas as pd

In [13]:
# take a look at one example training data (text)
train = pd.read_csv("train.csv")
train['text'][1],train['target'][1]

('Forest fire near La Ronge Sask. Canada', 1)

In [14]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [17]:
def make_csv(df):
  df1 = df.drop(columns=['id'])
  df1['text'] = df1.keyword.str.cat([df.text], sep=' ', na_rep='na')
  df1['label'] = df1.target
  df1.drop(columns=['keyword','location','target'],inplace=True)
  return df1

In [18]:
train1 = make_csv(train)
train1.head()

,text,label
0,na Our Deeds are the Reason of this #earthquak...,1
1,na Forest fire near La Ronge Sask. Canada,1
2,na All residents asked to 'shelter in place' a...,1
3,"na 13,000 people receive #wildfires evacuation...",1
4,na Just got sent this photo from Ruby #Alaska ...,1


In [19]:
train1.to_csv('train_text.csv')

In [20]:
# take a look at one example tresting data (text)
test = pd.read_csv("test.csv")
test['text'][1]

'Heard about #earthquake is different cities, stay safe everyone.'

In [21]:
def make_csv_test(df):
  df1 = df.drop(columns=['id'])
  df1['text'] = df1.keyword.str.cat([df.text], sep=' ', na_rep='na')
  df1.drop(columns=['keyword','location'],inplace=True)
  return df1

In [22]:
test1 = make_csv_test(test)
test1.head()

,text
0,na Just happened a terrible car crash
1,na Heard about #earthquake is different cities...
2,"na there is a forest fire at spot pond, geese ..."
3,na Apocalypse lighting. #Spokane #wildfires
4,na Typhoon Soudelor kills 28 in China and Taiwan


In [23]:
test1.to_csv('test_text.csv')

## Preprocess 预处理

In [24]:
from transformers import AutoTokenizer

In [25]:
#tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") #指定tokenizer类型
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [26]:
# def preprocess_function(examples):
#   return tokenizer(examples["text"], truncation=True)

def preprocess_function(example):
    text = example["text"]
    label = example["label"]
    tokenized_text = tokenizer(text, truncation=True)
    return {
        "text": text,
        "label": label,
        "input_ids": tokenized_text["input_ids"],
        "attention_mask": tokenized_text["attention_mask"]
    }

In [27]:
from sklearn.model_selection import train_test_split
train_train_data, train_test_data = train_test_split(train1, test_size=0.1, random_state=318)

In [28]:
train_examples = train_train_data.to_dict("records")
tokenized_train = [preprocess_function(example) for example in train_examples]

In [29]:
tokenized_train[1]

{'text': 'whirlwind Weather forecast for Thailand  A Whirlwind is coming ...2 september https://t.co/rUKjYjG9oQ',
 'label': 1,
 'input_ids': [101,
  1059,
  11961,
  2140,
  11101,
  4633,
  19939,
  2005,
  6504,
  1037,
  1059,
  11961,
  2140,
  11101,
  2003,
  2746,
  1012,
  1012,
  1012,
  1016,
  2244,
  16770,
  1024,
  1013,
  1013,
  1056,
  1012,
  2522,
  1013,
  21766,
  2243,
  3501,
  2100,
  3501,
  2290,
  2683,
  2080,
  4160,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [30]:
test_examples = train_test_data.to_dict("records")
tokenized_test = [preprocess_function(example) for example in test_examples]

In [31]:
tokenized_test[1]

{'text': 'traumatised @Jude_Mugabi not that all abortions get you traumatised. At times you are okay with the decision due to reasons like rape',
 'label': 0,
 'input_ids': [101,
  12603,
  7315,
  2098,
  1030,
  12582,
  1035,
  14757,
  28518,
  2025,
  2008,
  2035,
  11324,
  2015,
  2131,
  2017,
  12603,
  7315,
  2098,
  1012,
  2012,
  2335,
  2017,
  2024,
  3100,
  2007,
  1996,
  3247,
  2349,
  2000,
  4436,
  2066,
  9040,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [32]:
#create a batch of examples using DataCollatorWithPadding
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

In [33]:
import evaluate

accuracy = evaluate.load("accuracy") #加载评价指标方法

In [34]:
#create a function that passes your predictions and labels to compute to calculate the accuracy
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Train

In [35]:
id2label = {0: int("0"), 1: int("1")}
label2id = {int("0"):0,int("1"):1}

In [36]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    #"distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [37]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/StonecoldLi/my_model into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.6k/418M [00:00<?, ?B/s]

Download file runs/May24_01-00-10_dd70a836ab4a/events.out.tfevents.1684890082.dd70a836ab4a.554.0: 100%|#######…

Download file runs/May24_01-00-10_dd70a836ab4a/1684890082.619498/events.out.tfevents.1684890082.dd70a836ab4a.5…

Download file runs/May26_12-27-04_10071fa237eb/1685104134.425462/events.out.tfevents.1685104134.10071fa237eb.1…

...

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.409360,0.822835
2,No log,0.393219,0.845144
3,No log,0.394933,0.845144
4,No log,0.395536,0.839895


TrainOutput(global_step=432, training_loss=0.37847748509159795, metrics={'train_runtime': 435.4956, 'train_samples_per_second': 62.926, 'train_steps_per_second': 0.992, 'total_flos': 899885031543840.0, 'train_loss': 0.37847748509159795, 'epoch': 4.0})

## Inference

In [38]:
test1['text'][1]

'na Heard about #earthquake is different cities, stay safe everyone.'

In [39]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./my_model")
inputs = tokenizer(test['text'][1], return_tensors="pt")

In [40]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained("./my_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [41]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

1

In [42]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("./my_model")
model = AutoModelForSequenceClassification.from_pretrained("./my_model")

# 假设 test 是一个包含句子的 Series 对象
test_sentences = test1['text']

# 将句子列表转换为模型输入的批量格式
inputs = tokenizer(test_sentences.tolist(), padding=True, truncation=True, return_tensors="pt")
batch_size = 32 #怕爆显存

# 将输入放入设备（GPU 或 CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(device) for k, v in inputs.items()}

# 将模型移动到相同的设备上
model = model.to(device)

# 批量预测
with torch.no_grad():
    logits = model(**inputs).logits

# 获取预测结果
predicted_class_ids = logits.argmax(dim=1).tolist()

# 获取预测标签
predicted_labels = [model.config.id2label[pred] for pred in predicted_class_ids]

# 输出结果
'''
for label in predicted_labels:
    print(label)
'''


'\nfor label in predicted_labels:\n    print(label)\n'

In [43]:
test = pd.read_csv('test.csv')

In [44]:
pd.DataFrame({'id':test.id, 'target':predicted_labels}).set_index('id').to_csv('submission_bert-base-uncased_textkeyword_ver1.csv')

- 0.8345 